# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [32]:
# Dependencies and Setup
import geoviews as gv
import hvplot.pandas
import hvplot
import pandas as pd
import requests
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [2]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_search_data_to_load = ("../Vacation Search/WeatherPy_Vacation.csv")
vacation_df = pd.read_csv(vacation_search_data_to_load)

# Display sample data
vacation_df.head(5)

,City_ID,City,Country,Max Temp,Current Conditions,Lat,Lng,Hotel Name
0,0,Kapaa,US,80.58,clear sky,22.0752,-159.3190,Pono Kai Resort
1,1,Havelock,US,66.94,clear sky,34.8791,-76.9013,Quality Inn Havelock Hwy 70
2,2,Tabou,CI,78.10,scattered clouds,4.4230,-7.3528,hôtel le rochet
3,3,Hobart,AU,55.22,scattered clouds,-42.8794,147.3294,The Old Woolstore Apartment Motel
4,7,Rikitea,PF,72.28,overcast clouds,-23.1203,-134.9692,Chez Bianca & Benoit


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [3]:
# Configure the map
map_plot = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = (["City", "Country", "Current Conditions"])
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Current Conditions)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [4]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"] == "Cape Town"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Cape Town"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Hermanus"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Bredasdorp"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Port Elizabeth"]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [5]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end], axis=0)

# Display sample data
itinerary_df

,City_ID,City,Country,Max Temp,Current Conditions,Lat,Lng,Hotel Name
32,84,Cape Town,ZA,66.11,clear sky,-33.9258,18.4232,Townhouse Hotel
18,37,Hermanus,ZA,65.80,few clouds,-34.4187,19.2345,Aloe guest house
21,47,Bredasdorp,ZA,64.78,clear sky,-34.5322,20.0403,Victoria Hotel
75,194,Port Elizabeth,ZA,73.09,clear sky,-33.9180,25.5701,Waterford Hotel
32,84,Cape Town,ZA,66.11,clear sky,-33.9258,18.4232,Townhouse Hotel


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [6]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[['City', 'Lng', 'Lat']].copy()

# Display sample data
waypoints_df

,City,Lng,Lat
32,Cape Town,18.4232,-33.9258
18,Hermanus,19.2345,-34.4187
21,Bredasdorp,20.0403,-34.5322
75,Port Elizabeth,25.5701,-33.9180
32,Cape Town,18.4232,-33.9258


## Use GeoViews to create map that shows the four cities in the itinerary

In [23]:
# Configure the map plot by using the itineraty_df
waypoints_map = itinerary_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = 50,
    legend = 'right',
    color = 'City'
    )

In [24]:
# Display the route_map
waypoints_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [9]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [14]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': 'd2a0f6484aed442b94a8b1de808654b7',
 'waypoints': '-33.9258,18.4232|-34.4187,19.2345|-34.5322,20.0403|-33.918,25.5701|-33.9258,18.4232'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [15]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = requests.get(base_url, params=params)

# Convert the API response to JSON format
route_response = route_response.json()
route_response

{'features': [{'type': 'Feature',
   'properties': {'mode': 'drive',
    'waypoints': [{'location': [18.4232, -33.9258], 'original_index': 0},
     {'location': [19.2345, -34.4187], 'original_index': 1},
     {'location': [20.0403, -34.5322], 'original_index': 2},
     {'location': [25.5701, -33.918], 'original_index': 3},
     {'location': [18.4232, -33.9258], 'original_index': 4}],
    'units': 'metric',
    'distance': 1561194,
    'distance_units': 'meters',
    'time': 56432.263,
    'legs': [{'distance': 118591,
      'time': 4637.711,
      'steps': [{'from_index': 0,
        'to_index': 3,
        'distance': 158,
        'time': 9.492,
        'instruction': {'text': 'Drive southeast on Caledon Street.'}},
       {'from_index': 3,
        'to_index': 24,
        'distance': 390,
        'time': 32.097,
        'instruction': {'text': 'Turn left onto Buitenkant Street/M59. Continue on M59.'}},
       {'from_index': 24,
        'to_index': 30,
        'distance': 494,
        't

In [16]:
#Fetch the route's legs coordinates from the JSON reponse
legs = route_response['features'][0]['geometry']['coordinates'][0]
legs1 = route_response['features'][0]['geometry']['coordinates'][1]
legs2 = route_response['features'][0]['geometry']['coordinates'][2]
legs3 = route_response['features'][0]['geometry']['coordinates'][3]
all_legs = legs + legs1 + legs2 + legs3
all_legs

[[18.422919, -33.926062],
 [18.423419, -33.926432],
 [18.424143, -33.926975],
 [18.424191, -33.927011],
 [18.424246, -33.926958],
 [18.424363, -33.926852],
 [18.424752, -33.926502],
 [18.424801, -33.926451],
 [18.425158, -33.926097],
 [18.425239, -33.926021],
 [18.425328, -33.925939],
 [18.425819, -33.925439],
 [18.425982, -33.925287],
 [18.425999, -33.925272],
 [18.426023, -33.925239],
 [18.426032, -33.925206],
 [18.426033, -33.925174],
 [18.426024, -33.925115],
 [18.42597, -33.924961],
 [18.425935, -33.924843],
 [18.425938, -33.924767],
 [18.425962, -33.924705],
 [18.42624, -33.924393],
 [18.426292, -33.924309],
 [18.426368, -33.924175],
 [18.426457, -33.924206],
 [18.426587, -33.924246],
 [18.426706, -33.924283],
 [18.428482, -33.924882],
 [18.430244, -33.925457],
 [18.431324, -33.92583],
 [18.431934, -33.925989],
 [18.432191, -33.926029],
 [18.433424, -33.92609],
 [18.433557, -33.926103],
 [18.433688, -33.926137],
 [18.433811, -33.926185],
 [18.433914, -33.926237],
 [18.434037, -33

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [17]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for coordinate in all_legs:
    long = coordinate[0]
    lat = coordinate[1]
    
    longitude.append(long)
    latitude.append(lat)
   
print(longitude)


[18.422919, 18.423419, 18.424143, 18.424191, 18.424246, 18.424363, 18.424752, 18.424801, 18.425158, 18.425239, 18.425328, 18.425819, 18.425982, 18.425999, 18.426023, 18.426032, 18.426033, 18.426024, 18.42597, 18.425935, 18.425938, 18.425962, 18.42624, 18.426292, 18.426368, 18.426457, 18.426587, 18.426706, 18.428482, 18.430244, 18.431324, 18.431934, 18.432191, 18.433424, 18.433557, 18.433688, 18.433811, 18.433914, 18.434037, 18.434112, 18.434187, 18.434237, 18.434286, 18.434322, 18.434355, 18.434367, 18.434359, 18.434329, 18.434334, 18.43436, 18.4344, 18.434465, 18.434553, 18.434677, 18.434915, 18.435168, 18.435295, 18.435449, 18.435544, 18.435638, 18.436201, 18.436539, 18.436966, 18.437243, 18.437508, 18.437804, 18.438072, 18.438406, 18.438772, 18.439372, 18.439577, 18.439841, 18.440108, 18.440294, 18.440544, 18.44077, 18.440983, 18.441266, 18.44173, 18.442177, 18.442587, 18.443064, 18.443658, 18.444204, 18.444498, 18.44484, 18.445123, 18.445416, 18.445703, 18.446071, 18.446383, 18.446

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [18]:
# Create an empty DataFrame to store the steps' coordinates
route_df = pd.DataFrame()

# Add the steps' longitude and latitude from each step as columns to the DataFrame
route_df = pd.DataFrame({'Longitude': longitude, 'Latitude': latitude})
# Display sample data
route_df

,Longitude,Latitude
0,18.422919,-33.926062
1,18.423419,-33.926432
2,18.424143,-33.926975
3,18.424191,-33.927011
4,18.424246,-33.926958
...,...,...
23024,18.422668,-33.925761
23025,18.422662,-33.925805
23026,18.422668,-33.925848
23027,18.422685,-33.925889


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [35]:
from cartopy import crs
from geoviews import opts
import geoviews.tile_sources as gts
from bokeh.models import ColumnDataSource

# Configure the route path by using the GeoViews' Path function

# route_path = pd.DataFrame(route_df)
route_path = gv.Path(route_df).opts(width=700, height=450, color='red')


In [38]:
# Display a composed plot by using the route_map and route_path objects
# Configure the map
route_map = waypoints_map * route_path
# Display the map plot
hvplot.save(route_map, 'WeatherPy_Travel_Map.png')
route_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)
   .Path.I   :Path   [Longitude,Latitude]

     ---------------------------------------- 6.0/6.0 MB 34.9 MB/s eta 0:00:00
     ------------------------------------- 384.9/384.9 kB 23.4 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
